# Подгружаем нужные пакеты

In [1]:
import pandas as pd

import numpy as np

from datetime import datetime

import yfinance as yf

import matplotlib.pyplot as plt

import os

import math

import statistics

import xlsxwriter

# Скачиваем веса портфеля

In [10]:

df = pd.read_excel("Connect.xlsx", sheet_name=None)

sheets = pd.ExcelFile("Connect.xlsx").sheet_names

papers = []
for sheet in sheets:
    for comp in list(df[sheet]['comp'].values):
        papers.append(comp)

papers = list(np.unique(papers))

weights_min = {}
for comp in papers:
    w = []
    for sheet in sheets:
        if df[sheet][df[sheet]['comp'] == comp]['Weight'].empty:
            w.append(0)
        else:
            w.append(round(float(df[sheet][df[sheet]['comp'] == comp]['Weight']), 5))
    weights_min[comp] = w

## Добываем данные по валютам торгов

In [11]:
currencies = {}
tikers = yf.Tickers(papers)

for tiker in tikers.tickers:
    try:
        cur = tikers.tickers[tiker].history_metadata['currency']
        if cur != 'EUR':
            currencies[tiker] = cur+'EUR=X'
        else:
            currencies[tiker] = cur
    except:
        print(tiker)


In [12]:
valuta = yf.download(list(currencies.values()), start = "2017-03-01", end = "2023-06-30", interval='1mo')

[*********************100%***********************]  6 of 6 completed


## Качаем данные по бумагам

In [13]:
data = yf.download(papers, start = "2017-03-01", end = "2023-06-30", interval='1mo')

data = data.fillna(method = 'ffill')

data = data.fillna(0)

[*********************100%***********************]  22 of 22 completed


In [14]:
valuta = valuta['Adj Close']

In [79]:
currencies['AFC.L'] = 'GBPEUR=X'

## Выравниваем всю валюту к евро 

In [15]:
def currencie_adjustment(data, currencies, cur_data):
    new_data = pd.DataFrame()
    for asset in currencies:
        if currencies[asset] != 'EUR':
            new_data[asset] = data['Adj Close'][asset] * cur_data[currencies[asset]]
        else:
            new_data[asset] = data['Adj Close'][asset]
    return(new_data)

In [16]:
new_data = currencie_adjustment(data, currencies=currencies, cur_data = valuta)
new_data = new_data.fillna(method = 'ffill')

## Сам бэктест

In [82]:
def trade(i, weights, portfolio, month_counter, cash, data, comission, orders, total_comission):
    value = 0
    asset_value = {}
    for asset in portfolio:
        asset_value[asset] = portfolio[asset] * data.iloc[i][asset]
        value += asset_value[asset]

    for asset in portfolio:
                
        target_weight = weights[asset][month_counter]
        current_weight = asset_value[asset] / value
        if data.iloc[i][asset] != 0:
            if current_weight < target_weight:
                req_pos = math.floor((value + cash) * target_weight / data.iloc[i][asset])
                diff = req_pos - portfolio[asset]
                if abs(diff)*data.iloc[i][asset] <= cash:
                    orders.append([asset, diff, data.iloc[i][asset], data.iloc[i].name.date()])
                    portfolio[asset] = req_pos
                    cash -= (diff * data.iloc[i+2][asset]+3)
                    total_comission += 3

            elif current_weight > target_weight:
                req_pos = math.floor((value + cash) * target_weight / data.iloc[i][asset])
                diff = portfolio[asset] - req_pos
                orders.append([asset, -diff, data.iloc[i][asset], data.iloc[i].name.date()])
                portfolio[asset] = req_pos
                cash += diff * data.iloc[i+2][asset] - 3
                total_comission += 3
    return(portfolio, value, cash, orders, total_comission)
    
def backtest(data, cash, weights):

    comission = 0.005
    dates = data.index

    portfolio = {}
    for asset in weights:
        portfolio[asset] = 0

    prev_month = None
    first_date = None
    equity = [cash]
    orders = []

    d = []

    month_counter = 0
    total_comission = 0

    track_cash = []

    for i in range(len(dates)-2):

        month = data.iloc[i].name.month

        if prev_month is None:  # для первой даты в массиве
            sum = 0
            prev_month = month
            first_date = i
            print("Месяц изменился:", prev_month, "->", month)
            print("Первая дата года:", data.iloc[i].name.date())
            for asset in portfolio:
                if data.iloc[i][asset] != 0:
                    quantity = math.floor(cash*weights_min[asset][month_counter]/data.iloc[i+2][asset])
                    portfolio[asset] = quantity
                    orders.append([asset, quantity, data.iloc[i+2][asset], data.iloc[i].name.date()])
                    sum += portfolio[asset] * data.iloc[i+2][asset]
                    total_comission += 3
            cash = cash - sum*(1+comission) - 3
            d.append(data.iloc[i].name)


        elif prev_month != month:
            prev_month = month
            month_counter += 1
            portfolio, value, cash, orders, total_comission = trade(i = i, weights=weights, portfolio=portfolio, month_counter=month_counter, cash=cash, data=data, comission=comission, orders=orders, total_comission=total_comission)
            equity.append(value + cash)
            d.append(data.iloc[i].name)
        else:
            portfolio, value, cash, orders, total_comission = trade(i = i, weights=weights, portfolio=portfolio, month_counter=month_counter, cash=cash, data=data, comission=comission, orders=orders, total_comission=total_comission)
            equity.append(value + cash)
            d.append(data.iloc[i].name)
            
    return equity, orders, total_comission, portfolio, d

In [83]:
eq, ord, tc, portf, d = backtest(new_data, cash = 10000, weights = weights_min)

Месяц изменился: 3 -> 3
Первая дата года: 2017-03-01


In [84]:
dates = data.index
plt.figure(figsize=(15, 5))
plt.plot(dates[:-2], eq)

### Создание файла со статистикой и картинками

In [85]:
def create_plot(equity, banch, banchmark, adjusted = False):
    plt.ioff()
    if adjusted:
        plt.figure(figsize = (15,6))
        plt.plot(equity, color='red', label = 'portfolio')

        plt.plot(banch['Adj Close']/banch['Adj Close'][0], color='blue', label = banchmark)
        plt.xlabel('Time')
        plt.ylabel('Return')
        
        
        plt.legend()
        
        plot_name = banchmark+'_adj'
    else:
        fig, ax1 = plt.subplots(figsize=(15, 6))
           
        # Построение первого набора данных на первой оси
        ax1.plot(equity, color='red', label = 'portfolio')
        ax1.tick_params(axis='y', labelcolor='red')

        # Создание второй оси для второго набора данных
        ax2 = ax1.twinx()

        # Построение второго набора данных на второй оси
        ax2.plot(banch['Adj Close'], color='blue', label = banchmark)
        ax2.tick_params(axis='y', labelcolor='blue')
        plot_name = banchmark

        # Отvображение графика
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        lines = lines1 + lines2
        labels = labels1 + labels2
        ax1.legend(lines, labels, loc='upper left')
        plt.xlabel('Time')
        plt.ylabel('Return')
            
    
    plt.savefig(plot_name)

In [86]:
def create_stats(eq, banchmarks, start_cash, num_years, risk_free, orders, total_comission, filename):
    compaund_return = ((eq[-1]/start_cash)**(1/num_years) - 1) *100

    total_return = eq[-1]/start_cash - 1

    vol = np.std(eq)

    sharpe = (np.mean(np.array(eq)/start_cash - 1) - risk_free)/np.std(np.array(eq)/start_cash - 1)

    equity = pd.Series(eq, index = dates[:-2])
    
    return_portfolio = (equity / equity.shift(1)) - 1

    portfolio_value = equity
    drawdowns = np.zeros_like(portfolio_value)
    for i in range(1, len(portfolio_value)):
        drawdowns[i] = (portfolio_value[i] - np.max(portfolio_value[:i])) / np.max(portfolio_value[:i])

    # Calculate the maximum drawdown of the portfolio
    max_drawdown = np.min(drawdowns)

    max_drawdown_percent = max_drawdown * 100

    end_idx = np.where(drawdowns == max_drawdown)[0][-1]
    start_idx = np.argmax(portfolio_value[:end_idx])  

    orders_table = pd.DataFrame(orders, columns=['Tiker', 'Quantity', 'Price', 'Date'])

    general_results = pd.DataFrame([[compaund_return, total_return*100,vol, sharpe, max_drawdown_percent, dates[start_idx].date(), dates[end_idx].date(), total_comission, sum(abs(orders_table['Quantity'])*orders_table['Price']),len(orders), ]], 
                                   columns=[ 'Compaund return','Return','Std', 'Sharpe', 'Max Drawdown','Begin Drawdown','End Drawdown', 'Comisson fees', 'Volume','Num trades'])

    with pd.ExcelWriter(filename+'.xlsx', engine = 'xlsxwriter') as writer:   
        general_results.to_excel(writer, sheet_name = 'Metrix')  
        orders_table.to_excel(writer, sheet_name='Orders')

        worksheet = writer.sheets['Metrix']
        worksheet.write('M4', 'Dynamics adjusted to 1:')
        col = 'A'
        
        row_counter = 4
        for banchmark in banchmarks:

            banch = yf.download(banchmark, start = '2017-03-01', end = '2023-06-30', interval='1mo')
            market_return = (banch['Adj Close']/banch['Adj Close'].shift(1)) - 1

            total_banch_return = banch['Adj Close'][-1]/banch['Adj Close'][0] - 1

            result = return_portfolio[1:].loc[market_return[1:-2].index]

            beta = np.cov(result, market_return[1:-2])[0, 1] / np.var(market_return)
            
            Rp = result.mean()  # средняя доходность портфеля
            Rf = risk_free  # безрисковая ставка доходности
            Rm = market_return[1:].mean()

            alpha = Rp - beta * (Rm - Rf)

            te = np.std(result - market_return[1:-2])    
            ir = (total_return - total_banch_return)/te

            worksheet.write(col+str(row_counter), banchmark)
            banch_results = pd.DataFrame([[alpha, beta, total_banch_return*100, ir]], columns=[ 'Alpha', 'Beta', 'Total return of Banchmark', 'IR'])

            banch_results.to_excel(writer, sheet_name = 'Metrix', startrow = row_counter)

            row_counter += 3

            create_plot(equity = equity, banch=banch, banchmark = banchmark)

            create_plot(equity = equity/start_cash, banch=banch, banchmark = banchmark, adjusted = True)

            worksheet.insert_image(col+str(row_counter),banchmark+'.png', {"x_scale": 0.5, "y_scale": 0.5})

            worksheet.insert_image('M'+str(row_counter),banchmark+'_adj.png', {"x_scale": 0.5, "y_scale": 0.5})
            row_counter += 15


In [87]:
plt.clf()
create_stats(eq, banchmarks = ['^GSPC', 'ICLN','^STOXX', 'URTH', '^GSPE'], start_cash = 10000, num_years = 7, risk_free = 0.0237, orders=ord, total_comission = tc, filename = 'small_results_3')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


20% облиг


In [54]:
weights_20 = weights_min

for i in range(len(weights_min['NEE'])):
    if weights_20['BGRN'] != 0:
        weights_20['BGRN'][i] -= 0.1
        for asset in weights_20:
            weights_20[asset][i] += weights_20[asset][i]/0.55 * 0.1



In [55]:
eq, ord, tc, portf, d = backtest(new_data, cash = 10000, weights = weights_20)

Месяц изменился: 3 -> 3
Первая дата года: 2017-03-01


In [56]:
plt.clf
plt.ioff
create_stats(eq, banchmarks = ['^GSPC', 'ICLN','^STOXX', 'URTH', '^GSPE'], start_cash = 10000, num_years = 7, risk_free = 0.0237, orders=ord, total_comission = tc, filename='small_back_20')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [57]:
weights_10 = weights_20

for i in range(len(weights_min['300274.SZ'])):
    if weights_10['BGRN'] != 0:
        weights_10['BGRN'][i] -= 0.1
        for asset in weights_10:
            weights_10[asset][i] += weights_10[asset][i]/0.55 * 0.1

In [58]:
eq, ord, tc, portf, d = backtest(new_data, cash = 100000, weights = weights_10)
plt.clf
plt.ioff
create_stats(eq, banchmarks = ['^GSPC', 'ICLN','^STOXX', 'URTH', '^GSPE'], start_cash = 100000, num_years = 7, risk_free = 0.0237, orders=ord, total_comission = tc, filename='connect_back_10')

Месяц изменился: 3 -> 3
Первая дата года: 2017-03-01
[*********************100%***********************]  1 of 1 completed


/var/folders/5v/nn5t_5c94zs0qj28kd48537m0000gn/T/ipykernel_33346/1398889471.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax1 = plt.subplots(figsize=(15, 6))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [59]:
eq, ord, tc, portf, d = backtest(new_data, cash = 10000, weights = weights_10)
plt.clf
plt.ioff
create_stats(eq, banchmarks = ['^GSPC', 'ICLN','^STOXX', 'URTH', '^GSPE'], start_cash = 10000, num_years = 7, risk_free = 0.0237, orders=ord, total_comission = tc, filename='connect_back_10b_10k')

Месяц изменился: 3 -> 3
Первая дата года: 2017-03-01
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [50]:
eq, ord, tc, portf, d = backtest(new_data, cash = 10000, weights = weights_20)
plt.clf
plt.ioff
create_stats(eq, banchmarks = ['^GSPC', 'ICLN','^STOXX', 'URTH', '^GSPE'], start_cash = 10000, num_years = 7, risk_free = 0.0237, orders=ord, total_comission = tc, filename='connect_back_20b_10k')

Месяц изменился: 3 -> 3
Первая дата года: 2017-03-01
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [56]:
eq, ord, tc, portf, d = backtest(new_data, cash = 10000, weights = weights_min)
plt.clf
plt.ioff
create_stats(eq, banchmarks = ['^GSPC', 'ICLN','^STOXX', 'URTH', '^GSPE'], start_cash = 10000, num_years = 7, risk_free = 0.0237, orders=ord, total_comission = tc, filename='connect_back_30b_10k')

Месяц изменился: 3 -> 3
Первая дата года: 2017-03-01
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
